In [ ]:
import numpy as np
import argparse
import pandas as pd
import glob
import datetime
import matplotlib.pyplot as plt
import act
import os
import datetime as dt
import sys
from tempfile import TemporaryDirectory
from matplotlib import patheffects as pe
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import datetime as dt
import altair as alt
alt.data_transformers.enable('json')

# Set inputs

In [ ]:
USERNAME = os.getenv("ARM_USERNAME")
TOKEN = os.getenv("ARM_TOKEN")
SAIL_DATA_STREAM = 'gucdlrhiM1.b1'
SNR_THRESHOLD = 0.008
DATE_FORMAT = "%Y-%m-%d"
MAX_RANGE = 2000
dates = [
    dt.date(2023, 3, 3), # actual date of interest
    dt.date(2023, 3, 4), 
    dt.date(2023, 4, 16), # actual date of interest
    dt.date(2023, 4, 17),
    dt.date(2023, 5, 2), # actual date of interest
    dt.date(2023, 5, 3),
    dt.date(2023, 6, 9),  # actual date of interest
    dt.date(2023, 6, 10),
    dt.date(2023, 6, 11),  # actual date of interest
    dt.date(2023, 6, 12), 
]
dates_actual = [
    dt.date(2023, 3, 3),
    dt.date(2023, 4, 16),
    dt.date(2023, 5, 2),
    dt.date(2023, 6, 9),
    dt.date(2023, 6, 11),
]

# Download data

In [ ]:
output_path = f"/Users/elischwat/Development/data/sublimationofsnow/{SAIL_DATA_STREAM}/"

In [ ]:
print("Beginning data download...")
for date in dates:        
    act.discovery.download_arm_data(USERNAME, TOKEN, SAIL_DATA_STREAM, date.strftime('%Y%m%d'), date.strftime('%Y%m%d'), output=output_path)
print("Data download complete.")

print("Opening files...")
dl_rhi_files = glob.glob(output_path + '*cdf')

dl_rhi = act.io.arm.read_arm_netcdf(dl_rhi_files)
print("File opening complete.")

print("Converting to dataframe...")
src_rhi = dl_rhi.to_dataframe().reset_index()
print("Conversion complete.")

In [ ]:
src_rhi.to_parquet('src_rhi.parquet')

In [ ]:
ls -lah | grep parquet

In [ ]:
src_rhi = pd.read_parquet('src_rhi.parquet')

# Preprocess data

In [ ]:
# Convert time zone
src_rhi['time'] = src_rhi['time'].dt.tz_localize('UTC').dt.tz_convert('US/Mountain')
src_rhi['time'] = pd.to_datetime(src_rhi['time'].dt.tz_localize(None))
# Remove Extra
src_rhi = src_rhi[src_rhi['time'].dt.date.isin(dates_actual)]
# Shrink dataset by removing extra data
src_rhi = src_rhi.query(f"range < {MAX_RANGE}")
# Filter with SNR
src_rhi['SNR'] = src_rhi['intensity'] - 1
src_rhi.loc[src_rhi.eval(f'SNR < {SNR_THRESHOLD}'), 'radial_velocity'] = np.nan
# Remove extraneous scan data
src_rhi = src_rhi.query("elevation != -0.01")
# Add useful columns
src_rhi['date'] = src_rhi['time'].dt.date
src_rhi['hour'] = src_rhi['time'].dt.hour
src_rhi['minute'] = src_rhi['time'].dt.minute
src_rhi['second'] = src_rhi['time'].dt.second
src_rhi['time_beginning_of_hour'] = src_rhi['time'].apply(lambda dt: dt.replace(minute=0, second=0, microsecond=0))
# RHI: convert polar coordinates to rectangular coords with the radar at (0,0)
src_rhi['x'] = src_rhi['range']*np.cos(np.deg2rad(src_rhi['elevation']))
src_rhi['z'] = src_rhi['range']*np.sin(np.deg2rad(src_rhi['elevation']))

In [ ]:
# Correct radial velocities to horizontal velocities
src_rhi['horizontal_velocity'] = (- src_rhi['radial_velocity']*np.cos(np.deg2rad(src_rhi['elevation'])))

In [ ]:
VALLEY_INCLINE = 10
src_rhi['streamwise_velocity'] = src_rhi['horizontal_velocity'] / np.cos(np.deg2rad(VALLEY_INCLINE))

# Separate cross vally and along valley scans

In [ ]:
scan_azimuth_valley_wise = 149
scan_azimuth_valley_cross = 270

In [ ]:
# Split dataset into valley-wise and cross-valley RHI scans
valley_rhi_df = src_rhi[np.abs(src_rhi['azimuth'] - scan_azimuth_valley_wise) < 1]
xvalley_rhi_df = src_rhi[np.abs(src_rhi['azimuth'] - scan_azimuth_valley_cross) < 1]

# Label the 4 cross-valley scans that happen each hour
# We do this by defining the "hourly seconds" (second for a data point where 0 seconds is 
# at the beginning of the hour)
# and saying that all data from after 913 (after 15:13 mm:ss) and before 1113 (before 18:13 
# mm:ss) is the first scan, and so on - this may be imperfect
xvalley_rhi_df['hourly_seconds'] = xvalley_rhi_df.apply(lambda row: row['minute']*60 + row['second'], axis=1)
xvalley_rhi_df['hourly_scan_n'] = pd.cut(
    xvalley_rhi_df['hourly_seconds'],
    [913, 1113, 2713, 2913, 3599],
    labels=['15.00','18.00','45.00','48.00']
)

# Label the 4 valley-wise scans that happen each hour
# similarly to above
valley_rhi_df['hourly_seconds'] = valley_rhi_df.apply(lambda row: row['minute']*60 + row['second'], axis=1)
valley_rhi_df['hourly_scan_n'] = pd.cut(
    valley_rhi_df['hourly_seconds'],
    [22, 214, 1804, 2014, 3599],
    labels=['00.00','03.00','30.00','33.00']
)

# Extract profile time series at given upvalley/downvalley distances

Establish vertical bin spacing and sizing

In [ ]:
# bin z so we have high res near surface (<100m), low res above
z_bins = np.concatenate([
    np.linspace(0,100,11), 
    np.linspace(0,2000,41)[3:]
])
z_bins

### Extract data at a set horizontal distance from the lidar

In [ ]:
# PROFILE_OFFSET_HORIZONTAL = + 500
PROFILE_OFFSET_HORIZONTAL = - 250
SAMPLING_TOLERANCE = 75

src = valley_rhi_df[np.abs(valley_rhi_df['x'] - PROFILE_OFFSET_HORIZONTAL) < SAMPLING_TOLERANCE]
src.loc[:, 'scan_time'] = src.apply(lambda row : row['time_beginning_of_hour']+ dt.timedelta(minutes = float(row['hourly_scan_n'])), axis=1)
src['scan_time'] = src.apply(
    lambda row: row['scan_time'].replace(
        year = row['time'].year, 
        month = row['time'].month, 
        day = row['time'].day,
    ),
    axis=1
)
src = src[['scan_time', 'radial_velocity', 'horizontal_velocity', 'streamwise_velocity',  'z', 'x']]
src.loc[:, 'z_binned'] = pd.cut(src.z, bins=z_bins).apply(lambda bin: (bin.left + bin.right)/2).astype('float')

In [ ]:
src.scan_time.min()

In [ ]:
if PROFILE_OFFSET_HORIZONTAL > 0:
    x_axis_label = 'Downvalley velocity (m/s)'
else:
    x_axis_label = 'Upvalley velocity (m/s)'

### Plot profiles (for one case study)

In [ ]:
CASE_STUDY_MONTH = 6
CASE_STUDY_DAY = 9

#### Profile data, as-is

In [ ]:
chart_src = src[
    (src.scan_time.dt.month == CASE_STUDY_MONTH) &
    (src.scan_time.dt.day == CASE_STUDY_DAY)
].set_index('scan_time').groupby([
    pd.Grouper(freq='120Min'),
    'z_binned'
])[['horizontal_velocity']].median().reset_index()

chart_src.loc[:, 'time_of_day_str'] = chart_src['scan_time'].dt.strftime('%H%M')
chart_src['time_category'] = pd.cut(
    chart_src['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['morning (0000-0600)', 'day (0700-1500)', 'night (1600-2300)']
)
chart_src = chart_src.query("z_binned <= 600")

chart = (
    alt.Chart(chart_src).transform_calculate(rule='0').mark_rule().encode(x='rule:Q')
    +
    alt.Chart(
        chart_src
    ).mark_line().encode(
        alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title(x_axis_label),
        alt.Y("z_binned:Q"),
        alt.Color("time_of_day_str:O").scale(scheme='rainbow'),
        tooltip="time_of_day_str:O"
    ).properties(width = 200, height = 200)
).facet(
    column=alt.Column('time_category:O').sort(['morning', 'day', 'night']).title(None),
).properties(
    title = f"{chart_src.scan_time.dt.date.iloc[0]}"
).configure_legend(columns=2)

In [ ]:
chart

In [ ]:
chart_src = src[
    (src.scan_time.dt.month == CASE_STUDY_MONTH) &
    (src.scan_time.dt.day == CASE_STUDY_DAY)
].set_index('scan_time').groupby([
    pd.Grouper(freq='120Min'),
    'z_binned'
])[['streamwise_velocity']].median().reset_index()

chart_src.loc[:, 'time_of_day_str'] = chart_src['scan_time'].dt.strftime('%H%M')
chart_src['time_category'] = pd.cut(
    chart_src['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['morning (0000-0600)', 'day (0700-1500)', 'night (1600-2300)']
)
chart_src = chart_src.query("z_binned <= 600")

(
    alt.Chart(chart_src).transform_calculate(rule='0').mark_rule().encode(x='rule:Q')
    +
    alt.Chart(
        chart_src
    ).mark_line().encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title(x_axis_label),
        alt.Y("z_binned:Q"),
        alt.Color("time_of_day_str:O").scale(scheme='rainbow'),
        tooltip="time_of_day_str:O"
    ).properties(width = 200, height = 200)
).facet(
    column=alt.Column('time_category:O').sort(['morning', 'day', 'night']).title(None),
).properties(
    title = f"{chart_src.scan_time.dt.date.iloc[0]}"
).configure_legend(columns=2)

In [ ]:
chart.save(
    f"dl_profile_chart_{str(chart_src.scan_time.dt.date.iloc[0])}_{PROFILE_OFFSET_HORIZONTAL}.png"
)

#### Plot a profile normalized by the mean (remove synoptic influences)

In [ ]:
chart = alt.Chart(
    chart_src.groupby('z_binned')[['streamwise_velocity']].median().reset_index()
).mark_line().encode(
    alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title(x_axis_label),
    alt.Y("z_binned:Q"),
).properties(width = 100, height = 100)

In [ ]:
chart.save(
    f"dl_profile_chart_{str(chart_src.scan_time.dt.date.iloc[0])}_{PROFILE_OFFSET_HORIZONTAL}_mean.png",
    ppi=150
)

In [ ]:
normalized_profile = chart_src.groupby('z_binned')['streamwise_velocity'].median()
normalized_timeseries = chart_src.groupby(['scan_time']).apply(lambda df: df.set_index('z_binned')['streamwise_velocity'] - normalized_profile).melt(ignore_index = False).reset_index().rename(columns={'value': 'streamwise_velocity'})
normalized_timeseries.loc[:, 'time_of_day_str'] = normalized_timeseries['scan_time'].dt.strftime('%H%M')
normalized_timeseries['time_category'] = pd.cut(
    normalized_timeseries['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['morning (0000-0600)', 'day (0700-1500)', 'night (1600-2300)']
)
normalized_timeseries = normalized_timeseries.query("z_binned > 10")

In [ ]:
chart = plot = (
    alt.Chart(normalized_timeseries).transform_calculate(rule='0').mark_rule().encode(x='rule:Q')
    +
    alt.Chart(
        normalized_timeseries
    ).mark_line().encode(
        alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title(x_axis_label),
        alt.Y("z_binned:Q"),
        alt.Color("time_of_day_str:O").scale(scheme='rainbow'),
        tooltip="time_of_day_str:O"
    ).properties(width = 200, height = 200)
).facet(
    column=alt.Column('time_category:O').sort(['morning', 'day', 'night']).title(None),
).properties(
    title = f"{chart_src.scan_time.dt.date.iloc[0]}"
).configure_legend(columns=2)


In [ ]:
chart

In [ ]:
chart.save(
    f"dl_profile_chart_{str(normalized_timeseries.scan_time.dt.date.iloc[0])}_{PROFILE_OFFSET_HORIZONTAL}_normalized.png"
)

# Extract profile at a given time, at multiple upvalley/downvalley distances

for 20230416, 

use average 0200-0400

In [70]:
spatial_src = valley_rhi_df[
    (valley_rhi_df['time'] >= "20230416 0200") &
    (valley_rhi_df['time'] <= "20230416 0400")
]

PROFILE_OFFSET_HORIZONTAL_LIST = [-1500, -1000, -500, -250, -125, 125, 250, 500, 1000, 1500]
SAMPLING_TOLERANCE = 50

df_list = []
for x_offset in PROFILE_OFFSET_HORIZONTAL_LIST:
    df = spatial_src[np.abs(spatial_src['x'] - x_offset) < SAMPLING_TOLERANCE]
    df['x_offset'] = x_offset
    if x_offset > 0:
        df['radial_velocity'] = - df['radial_velocity']
    df_list.append(df)
spatial_src_df = pd.concat(df_list)

spatial_src_df.loc[:, 'z_binned'] = pd.cut(spatial_src_df.z, bins=z_bins).apply(lambda bin: (bin.left + bin.right)/2).astype('float')
spatial_src_df = spatial_src_df.groupby(['z_binned', 'x_offset'])[['horizontal_velocity', 'radial_velocity', 'streamwise_velocity']].median().reset_index()
spatial_src_df = spatial_src_df[spatial_src_df.z_binned <= 600]

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_1855/2373258296.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x_offset'] = x_offset
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_1855/2373258296.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x_offset'] = x_offset
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_1855/2373258296.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [71]:
plot_src = spatial_src_df.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        # "Upvalley velocity (m/s)"
        ""
    ).scale(domain=[-4,0], clamp=True).axis(values=[-3,-2,-1,0], domain=True),
    alt.Y("z_binned_adj:Q"),
    # alt.Color("x_offset:O").scale(scheme='spectral', zero=False),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [72]:
plot_src = spatial_src_df.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        # "Upvalley velocity (m/s)"
        ""
    ).scale(domain=[-4,0], clamp=True).axis(values=[-3,-2,-1,0], domain=True),
    alt.Y("z_binned:Q"),
    # alt.Color("x_offset:O").scale(scheme='spectral', zero=False),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [73]:
plot_src = spatial_src_df.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
z_chart = alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Horizontal velocity') &\
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Radial velocity')  &\
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Streamwise velocity')

In [74]:
plot_src = spatial_src_df.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
z_adj_chart = alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned_adj:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Horizontal velocity') &\
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned_adj:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Radial velocity')  &\
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned_adj:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Streamwise velocity')

In [75]:

z_chart | z_adj_chart

alt.HConcatChart(...)

for 20230416, 

use average 1200-1400

<!-- for 20230611, 

use average 0200-0400, and 1200-1400 -->

In [82]:
spatial_src = valley_rhi_df[
    (valley_rhi_df['time'] >= "20230416 1200") &
    (valley_rhi_df['time'] <= "20230416 1400")
]

PROFILE_OFFSET_HORIZONTAL_LIST = [-1500, -1000, -500, -250, -125, 125, 250, 500, 1000, 1500]
SAMPLING_TOLERANCE = 50

df_list = []
for x_offset in PROFILE_OFFSET_HORIZONTAL_LIST:
    df = spatial_src[np.abs(spatial_src['x'] - x_offset) < SAMPLING_TOLERANCE]
    df['x_offset'] = x_offset
    if x_offset > 0:
        df['radial_velocity'] = - df['radial_velocity']
    df_list.append(df)
spatial_src_df = pd.concat(df_list)

spatial_src_df.loc[:, 'z_binned'] = pd.cut(spatial_src_df.z, bins=z_bins).apply(lambda bin: (bin.left + bin.right)/2).astype('float')
spatial_src_df = spatial_src_df.groupby(['z_binned', 'x_offset'])[['horizontal_velocity', 'radial_velocity', 'streamwise_velocity']].median().reset_index()
spatial_src_df = spatial_src_df[spatial_src_df.z_binned <= 600]

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_1855/1679769692.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x_offset'] = x_offset
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_1855/1679769692.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x_offset'] = x_offset
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_1855/1679769692.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [86]:
plot_src = spatial_src_df.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        # "Upvalley velocity (m/s)"
        ""
    ).scale(domain=[-5,5], clamp=True),
    alt.Y("z_binned_adj:Q"),
    # alt.Color("x_offset:O").scale(scheme='spectral', zero=False),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [88]:
plot_src = spatial_src_df.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').title(
        # "Upvalley velocity (m/s)"
        ""
    ),
    alt.Y("z_binned:Q"),
    # alt.Color("x_offset:O").scale(scheme='spectral', zero=False),
    alt.Facet("x_offset:O", spacing=-80).title('Along-valley distance from lidar')
).properties(width = 100, height = 200).resolve_scale(x='shared').configure_view(strokeWidth=0).configure_axis(grid=False)

alt.Chart(...)

In [89]:
plot_src = spatial_src_df.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
z_chart = alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Horizontal velocity') &\
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Radial velocity')  &\
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Streamwise velocity')

In [90]:
plot_src = spatial_src_df.query("x_offset < 1500").query("z_binned > 15").query("z_binned < 1500")
plot_src['z_binned_adj'] = plot_src['z_binned'] + plot_src['x_offset']*np.tan(np.deg2rad(10))
z_adj_chart = alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("horizontal_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned_adj:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Horizontal velocity') &\
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("radial_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned_adj:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Radial velocity')  &\
alt.Chart(
    plot_src
).mark_line(strokeWidth=4).encode(
    alt.X("streamwise_velocity:Q").sort('-y').scale(domain=[-6,6], clamp=True).title("Upvalley velocity (m/s)"),
    alt.Y("z_binned_adj:Q"),
    alt.Color("x_offset:O").scale(scheme='spectral'),
).properties(width = 450, height = 200, title='Streamwise velocity')

In [91]:

z_chart | z_adj_chart

alt.HConcatChart(...)